### Install requirements & import libraries

In [1]:
!pip3 install imbalanced-learn

In [2]:
!pip3 install keras

In [3]:
import pandas as pd
import numpy as np

from imblearn.over_sampling import SMOTE, RandomOverSampler

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.utils import class_weight

import re

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, LSTM, Embedding, Input, GlobalMaxPool1D
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import regularizers

Using TensorFlow backend.


### Data Preprocessing

#### Load data

In [5]:
toxic = pd.read_csv('train_cleaned.csv')

In [6]:
toxic.describe(include='all')

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_tokenize,comment_text_tokenize_stemmed,comment_text_clean
count,159571,159571,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571,159571,159521
unique,159571,159305,NaN,NaN,NaN,NaN,NaN,NaN,158250,158225,157648
top,90f77cb80a1f7f0e,jun utc,NaN,NaN,NaN,NaN,NaN,NaN,['january'],['januari'],thank experi wikipedia test work revert remov ...
freq,1,11,NaN,NaN,NaN,NaN,NaN,NaN,21,21,22
mean,NaN,NaN,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805,NaN,NaN,NaN
std,NaN,NaN,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420,NaN,NaN,NaN
min,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
25%,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
50%,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
75%,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN


#### Drop NA

In [7]:
toxic.dropna(axis=0, inplace=True)

#### Train-Test Split

In [8]:
x_train, x_test, y_train, y_test = train_test_split(toxic.loc[:,'comment_text_clean'], toxic.iloc[:,2:8], test_size = .2, random_state = 43)

#### Create word corpus (TF-IDF)

In [9]:
max_features = 2000

In [10]:
#TF-IDF Vectors as features

# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=max_features)
tfidf_vect.fit(x_train)
x_train_tfidf =  tfidf_vect.transform(x_train)
x_test_tfidf =  tfidf_vect.transform(x_test)

In [11]:
x_train_tfidf_os_all = []
y_train_tfidf_os_all = []


for i in range(6):   
    sm_tfidf = RandomOverSampler(random_state=40)
    x_train_tfidf_os, y_train_tfidf_os = sm_tfidf.fit_resample(x_train_tfidf, y_train.iloc[:,i])
    x_train_tfidf_os_all.append(x_train_tfidf_os)
    y_train_tfidf_os_all.append(y_train_tfidf_os)

### Neural Network Model

In [12]:
data_dim = max_features
timesteps = 1
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, data_dim), return_sequences=True))
  
model.add(Dropout(0.5))

model.add(Flatten()) 

model.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.0022)))
model.add(Dropout(0.5))


model.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.0022)))
model.add(Dropout(0.5))

model.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.0022)))
model.add(Dropout(0.5))

model.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.0022)))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 64)             528640    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
__________

In [15]:
prediction_train = []
prediction_test = []

In [16]:
x_test_tfidf = x_test_tfidf.toarray().reshape(x_test_tfidf.shape[0], 1, x_test_tfidf.shape[1])

In [17]:
for i in range(6):
    callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='best_model' + str(i) +'2000.h5', monitor='val_loss', save_best_only=True)]
    
    x_train_tfidf_os = x_train_tfidf_os_all[i]
    x_train_tfidf_os = x_train_tfidf_os.toarray().reshape(x_train_tfidf_os.shape[0], 1, x_train_tfidf_os.shape[1])
    history = model.fit(x_train_tfidf_os, y_train_tfidf_os_all[i],
              batch_size=128, epochs=50, callbacks=callbacks,
              verbose=1,
              validation_split=0.1)
    model.save('my_model' + str(i) +'2000.h5')
    prediction_train.append(model.predict_proba(x_train_tfidf.toarray().reshape(x_train_tfidf.shape[0], 1, x_train_tfidf.shape[1])))
    prediction_test.append(model.predict_proba(x_test_tfidf))

Train on 207637 samples, validate on 23071 samples
Epoch 1/50
207637/207637 [==============================] - 12s 56us/step - loss: 0.3696 - acc: 0.8660 - val_loss: 0.2777 - val_acc: 0.8918
Epoch 2/50
207637/207637 [==============================] - 9s 44us/step - loss: 0.2662 - acc: 0.9054 - val_loss: 0.3181 - val_acc: 0.8652
Epoch 3/50
207637/207637 [==============================] - 9s 44us/step - loss: 0.2504 - acc: 0.9086 - val_loss: 0.2697 - val_acc: 0.8786
Epoch 4/50
207637/207637 [==============================] - 9s 44us/step - loss: 0.2407 - acc: 0.9104 - val_loss: 0.2488 - val_acc: 0.8905
Epoch 5/50
207637/207637 [==============================] - 9s 44us/step - loss: 0.2331 - acc: 0.9130 - val_loss: 0.2562 - val_acc: 0.8778
Epoch 6/50
207637/207637 [==============================] - 9s 44us/step - loss: 0.2271 - acc: 0.9139 - val_loss: 0.2562 - val_acc: 0.8806
Train on 227395 samples, validate on 25267 samples
Epoch 1/50
227395/227395 [==============================] - 10s

In [18]:
prediction_submission_array = np.asarray(prediction_test).reshape(6, 31905).transpose()

In [19]:
proba = pd.DataFrame(data=prediction_submission_array,columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate'], index=x_test.index.values.tolist())

In [20]:
proba.to_csv('submission_2000_max_feature.csv', index=True)

In [30]:
f1_train = []
cutoff = []
for i in range(6):
    best_f1 = 0
    best_val = 0
    for val in np.arange(0,1,0.001):
        if f1_score(y_train.iloc[:,i], prediction_train[i] > val) > best_f1:
            best_f1 = f1_score(y_train.iloc[:,i], prediction_train[i] > val)
            best_val = val
    f1_train.append(best_f1)
    cutoff.append(best_val)

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [31]:
f1_train

[0.7911528674887454,
 0.5366276549631557,
 0.8583448275862069,
 0.8708240534521158,
 0.7718204488778055,
 0.7387826086956522]

In [32]:
cutoff

[0.748, 0.967, 0.887, 0.994, 0.88, 0.974]

In [33]:
f1_test = [f1_score(y_test.iloc[:,i] , prediction_test[i] > cutoff[i]) for i in range(6)]

In [34]:
f1_test

[0.7345302981724912,
 0.4293577981651376,
 0.784077892325315,
 0.3057324840764331,
 0.6951111111111111,
 0.5032258064516129]

In [35]:
roc_auc_train = [roc_auc_score(y_train.iloc[:,i], prediction_train[i]) for i in range(6)]

In [36]:
roc_auc_test = [roc_auc_score(y_test.iloc[:,i], prediction_test[i]) for i in range(6)]

In [37]:
roc_auc_train

[0.9791480961754042,
 0.992160191125364,
 0.995378190586399,
 0.9995809008060695,
 0.990819916269545,
 0.9975233431935068]

In [38]:
roc_auc_test

[0.9537939303292271,
 0.9721864933712461,
 0.9706205905651881,
 0.9278813179276142,
 0.9623815887314571,
 0.9460900188142216]